In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import make_scorer
from sklearn.feature_selection import SelectFromModel
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import precision_score, recall_score, f1_score


# Read the CSV file
pf = pd.read_csv("/content/cirrhosis.csv")

pf.head(5)




,ID,Age,N_Days,Status,Drug,Sex,Ascites,Hepatomegaly,Spiders,Edema,...,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Stage3
0,1,21464,400,D,D-penicillamine,F,Y,Y,Y,Y,...,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4,0
1,2,20617,4500,C,D-penicillamine,F,N,Y,Y,N,...,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3,1
2,3,25594,1012,D,D-penicillamine,M,N,N,N,S,...,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4,0
3,4,19994,1925,D,D-penicillamine,F,N,Y,Y,S,...,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4,0
4,5,13918,1504,CL,Placebo,F,N,Y,Y,N,...,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3,1


In [26]:
pf.describe()

,ID,Age,N_Days,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage3
count,418.000000,418.000000,418.000000,418.000000,284.000000,418.000000,310.000000,312.000000,312.000000,282.000000,407.000000,418.000000,418.000000
mean,209.500000,18533.351675,1917.782297,3.220813,369.510563,3.497440,97.648387,1982.655769,122.556346,124.702128,257.024570,10.692344,0.373206
std,120.810458,3815.845055,1104.672992,4.407506,231.944545,0.424972,85.613920,2140.388824,56.699525,65.148639,98.325585,1.172052,0.484236
min,1.000000,9598.000000,41.000000,0.300000,120.000000,1.960000,4.000000,289.000000,26.350000,33.000000,62.000000,1.000000,0.000000
25%,105.250000,15644.500000,1092.750000,0.800000,249.500000,3.242500,41.250000,871.500000,80.600000,84.250000,188.500000,10.000000,0.000000
50%,209.500000,18628.000000,1730.000000,1.400000,309.500000,3.530000,73.000000,1259.000000,114.700000,108.000000,251.000000,10.600000,0.000000
75%,313.750000,21272.500000,2613.500000,3.400000,400.000000,3.770000,123.000000,1980.000000,151.900000,151.000000,318.000000,11.100000,1.000000
max,418.000000,28650.000000,4795.000000,28.000000,1775.000000,4.640000,588.000000,13862.400000,457.250000,598.000000,721.000000,18.000000,1.000000


In [27]:
pf.shape

(418, 21)

In [28]:
pf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             418 non-null    int64  
 1   Age            418 non-null    int64  
 2   N_Days         418 non-null    int64  
 3   Status         418 non-null    object 
 4   Drug           312 non-null    object 
 5   Sex            418 non-null    object 
 6   Ascites        312 non-null    object 
 7   Hepatomegaly   312 non-null    object 
 8   Spiders        312 non-null    object 
 9   Edema          418 non-null    object 
 10  Bilirubin      418 non-null    float64
 11  Cholesterol    284 non-null    float64
 12  Albumin        418 non-null    float64
 13  Copper         310 non-null    float64
 14  Alk_Phos       312 non-null    float64
 15  SGOT           312 non-null    float64
 16  Tryglicerides  282 non-null    float64
 17  Platelets      407 non-null    float64
 18  Prothrombi

In [29]:
pf.drop_duplicates(inplace = True) #no dupes

pf.shape

(418, 21)

In [30]:
pf.isna().sum()

ID                 0
Age                0
N_Days             0
Status             0
Drug             106
Sex                0
Ascites          106
Hepatomegaly     106
Spiders          106
Edema              0
Bilirubin          0
Cholesterol      134
Albumin            0
Copper           108
Alk_Phos         106
SGOT             106
Tryglicerides    136
Platelets         11
Prothrombin        0
Stage              6
Stage3             0
dtype: int64

In [31]:
#filling the null values with the median
pf = pf.fillna(pf.median(numeric_only=True))
pf.isna().sum()

ID                 0
Age                0
N_Days             0
Status             0
Drug             106
Sex                0
Ascites          106
Hepatomegaly     106
Spiders          106
Edema              0
Bilirubin          0
Cholesterol        0
Albumin            0
Copper             0
Alk_Phos           0
SGOT               0
Tryglicerides      0
Platelets          0
Prothrombin        0
Stage              6
Stage3             0
dtype: int64

In [32]:
categorical_columns = ['Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

pf.dropna(subset=categorical_columns, inplace=True)

pf.isna().sum().astype(int)

ID               0
Age              0
N_Days           0
Status           0
Drug             0
Sex              0
Ascites          0
Hepatomegaly     0
Spiders          0
Edema            0
Bilirubin        0
Cholesterol      0
Albumin          0
Copper           0
Alk_Phos         0
SGOT             0
Tryglicerides    0
Platelets        0
Prothrombin      0
Stage            0
Stage3           0
dtype: int64

In [33]:
pf.head(5)

,ID,Age,N_Days,Status,Drug,Sex,Ascites,Hepatomegaly,Spiders,Edema,...,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Stage3
0,1,21464,400,D,D-penicillamine,F,Y,Y,Y,Y,...,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4,0
1,2,20617,4500,C,D-penicillamine,F,N,Y,Y,N,...,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3,1
2,3,25594,1012,D,D-penicillamine,M,N,N,N,S,...,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4,0
3,4,19994,1925,D,D-penicillamine,F,N,Y,Y,S,...,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4,0
4,5,13918,1504,CL,Placebo,F,N,Y,Y,N,...,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3,1


In [34]:
X =pf[['Age','N_Days','Ascites', 'Hepatomegaly','Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT',  'Tryglicerides', 'Platelets',  'Prothrombin' ]]
y =pf['Stage3 ']
X.head(5)

,Age,N_Days,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin
0,21464,400,Y,Y,Y,Y,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2
1,20617,4500,N,Y,Y,N,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6
2,25594,1012,N,N,N,S,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0
3,19994,1925,N,Y,Y,S,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3
4,13918,1504,N,Y,Y,N,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9


In [35]:
y = pd.get_dummies(y)

In [36]:
X = pd.get_dummies(X)

X.head(5)

,Age,N_Days,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Ascites_N,Ascites_Y,Hepatomegaly_N,Hepatomegaly_Y,Spiders_N,Spiders_Y,Edema_N,Edema_S,Edema_Y
0,21464,400,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,False,True,False,True,False,True,False,False,True
1,20617,4500,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,True,False,False,True,False,True,True,False,False
2,25594,1012,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,True,False,True,False,True,False,False,True,False
3,19994,1925,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,True,False,False,True,False,True,False,True,False
4,13918,1504,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,True,False,False,True,False,True,True,False,False


In [37]:
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

X_train.shape


(218, 20)

In [38]:
X_test.shape

(94, 20)

In [39]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(218, 20)
(94, 20)
(218, 2)
(94, 2)


In [40]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_y_pred = lr.predict(X_test)

print("Score on training set: {:.3f}".format(lr.score(X_train, y_train)))
print("Score on test set: {:.3f}".format(lr.score(X_test, y_test)))

Score on training set: 0.178
Score on test set: -0.053


In [41]:
def get_metrics(y_true, y_pred):
    # MAE: the average of the absolute differences between the actual values and the predicted values
    mae = mean_absolute_error(y_true, y_pred)

    # MSE: the average of the squared differences between the actual values and the predicted values
    mse = mean_squared_error(y_true, y_pred)

    # RMSE: the square root of MSE
    rmse = mean_squared_error(y_true, y_pred, squared=False)

    # return all three metrics

    return mae, mse, rmse

lr_metrics = get_metrics(y_test, lr_y_pred)
results = pd.DataFrame({
    'Model': ['Linear Regression'],  # model names
    'MAE': [lr_metrics[0]],  # MAE values for each model
    'MSE': [lr_metrics[1]],  # MSE values for each model
    'RMSE': [lr_metrics[2]] # RMSE values for each model
})

# display results
results

,Model,MAE,MSE,RMSE
0,Linear Regression,0.467883,0.260243,0.51014


In [42]:
# Use cross_val_score with regular cross-validation (e.g., k-fold)
k = 5  # Number of folds
lr = LinearRegression()  # Assuming lr is already defined



# Mean Squared Error (MSE)
mse_scores_lr = -cross_val_score(lr, X, y, cv=k, scoring='neg_mean_squared_error')
print("\nMean Squared Errors for each fold:", mse_scores_lr)

# Root Mean Squared Error (RMSE)
rmse_scores_lr = np.sqrt(mse_scores_lr)
print("\nRMSE for each fold:", rmse_scores_lr)

# Mean Absolute Error (MAE)
mae_scores_lr = -cross_val_score(lr, X, y, cv=k, scoring='neg_mean_absolute_error')
print("\nMean Absolute Error for each fold:", mae_scores_lr)

# Create a DataFrame to store the results
results = pd.DataFrame({
    'Model': ['Linear Regression', 'CV Linear Regression'],  # Model names
    'MAE': [lr_metrics[0], mae_scores_lr.mean()],  # MAE values for each model
    'MSE': [lr_metrics[1], mse_scores_lr.mean()],  # MSE values for each model
    'RMSE': [lr_metrics[2], rmse_scores_lr.mean()]  # RMSE values for each model
})

results



Mean Squared Errors for each fold: [0.19506949 0.21299076 0.24198861 0.24439749 0.25460403]

RMSE for each fold: [0.44166671 0.46150922 0.49192338 0.49436574 0.50458303]

Mean Absolute Error for each fold: [0.3970695  0.41127867 0.45936268 0.45101962 0.46809591]


,Model,MAE,MSE,RMSE
0,Linear Regression,0.467883,0.260243,0.51014
1,CV Linear Regression,0.437365,0.229810,0.47881


Gradient Discent

In [46]:
# Assuming X_train contains categorical variables that need to be encoded
from sklearn.preprocessing import OneHotEncoder

# Initialize OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Fit and transform the categorical columns in X_train
X_train_encoded = encoder.fit_transform(X_train)

# Transform X_test using the same encoder
X_test_encoded = encoder.transform(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [48]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming your dataset is loaded into a DataFrame called 'pf'
# Extract features and target variable
X = pf.drop(columns=['Stage3 '])
y = pf['Stage3 ']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess the data (scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the SGDRegressor model
sgd_regressor = SGDRegressor(max_iter=1000, alpha=0.0001, learning_rate='constant', eta0=0.01, random_state=42)

# Train the model
sgd_regressor.fit(X_train_scaled, y_train)

# Make predictions
y_pred = sgd_regressor.predict(X_test_scaled)

# Evaluate the model (e.g., calculate Mean Squared Error)
mse = ((y_pred - y_test) ** 2).mean()
print("Mean Squared Error:", mse)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ValueError: could not convert string to float: 'D'

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Your code continues...
# Calculate helper metrics
accuracy = model.score(X_test, y_test)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


In [ ]:
import matplotlib.pyplot as plt

# Plot actual versus predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='blue', alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted Values (SGDRegressor)')
plt.show()


In [ ]:

def gradient_descent_mlr(X, y, weights, eta, epsilon, max_iter):
    m = len(X)
    num_features = X.shape[1]
    costs = []

    for _ in range(max_iter):
        # Compute predicted values
        y_pred = np.dot(X, weights)

        # Compute error
        error = y_pred - y

        # Update weights
        gradient = np.dot(X.T, error) / m
        weights -= eta * gradient

        # Compute cost (mean squared error)
        cost = np.sum(error**2) / (2 * m)
        costs.append(cost)

        # Check convergence
        if cost < epsilon:
            break

    return weights, costs

# Extract features and target variable
X = np.array(pf[['Age', 'N_Days', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Platelets', 'Prothrombin']])
y = pf['Stage3 ']  # Remove the extra space

# Initialize weights (coefficients)
num_features = X.shape[1]
weights = np.zeros(num_features)

# Hyperparameters
eta = 0.01  # Learning rate
epsilon = 0.001  # Convergence threshold
max_iter = 1000

# Run gradient descent
final_weights, costs = gradient_descent_mlr(X, y, weights, eta, epsilon, max_iter)

# Print final coefficients
print(f"Final model coefficients: {final_weights}")

# Plot cost vs. iteration
plt.figure(dpi=100)
plt.xlabel('Iteration')
plt.ylabel('Cost')
plt.plot(costs)
plt.show()
